# PD @ Academia Ática
# Programación Funcional
## Correspondencia de Curry-Howard & Tipos Algebraicos de Datos

In [ ]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

In [ ]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)

## Ejercicios

$\{q \rightarrow \neg p, r \vee p\} \vdash \neg r \rightarrow \neg q$

In [ ]:
def proof[P, Q, R]: (Q => Not[P]) => Either[R, P] => (Not[R] => Not[Q]) =
    qnp => rop => rop match {
        case Left(r) =>
            nr => (q => nr(r))
        case Right(p) =>
            nr => (q => qnp(q)(p))
    }

$\{p \rightarrow q \rightarrow r, \neg s \vee p, q\} \vdash s \rightarrow r$

In [ ]:
def proof[P, Q, R, S](piqir: P => Q => R, nsop: Either[Not[S], P], q: Q): S => R =
    nsop match {
        case Left(ns) =>
            s => ns(s)
        case Right(p) =>
            s => piqir(p)(q)
    }

$\{s \rightarrow t \rightarrow u, u \rightarrow \neg u, (v \rightarrow s) \wedge (p \rightarrow t)\} \vdash v \rightarrow \neg p$

In [ ]:
def proof[P, S, T, U, V](sitiu: S => T => U, uinu: U => Not[U], uisapit: (V => S, P => T)): V => Not[P] =
    v => p => {
        val u = sitiu(uisapit._1(v))(uisapit._2(p))
        uinu(u)(u)
    }

$\{p \vee q, p \rightarrow r, q \rightarrow s \wedge r\} \vdash r$

In [ ]:
def proof[P, Q, R, S](poq: Either[P, Q], pir: P => R, qisar: Q => (S, R)): R =
    poq match {
        case Left(p) =>
            pir(p)
        case Right(q) =>
            qisar(q)._2
    }

$\{p \vee q, p \rightarrow r \vee \neg t, q \rightarrow s \wedge r\} \vdash t \rightarrow r$

In [ ]:
def proof[P, Q, R, S, T](poq: Either[P, Q], piront: P => Either[R, Not[T]], pisor: Q => (S, R)): T => R =
    t => poq match {
        case Left(p) =>
            piront(p) match {
                case Left(r) =>
                    r
                case Right(nt) =>
                    nt(t)
            }
        case Right(q) =>
            pisor(q)._2
    }

$\{p \vee q \vee r, \neg p \vee s, s \rightarrow r\} \vdash \neg q \rightarrow r$

In [ ]:
def proof[P, Q, R, S](poqor: Either[P, Either[Q, R]], npos: Either[Not[P], S], sir: S => R): Not[Q] => R =
    (poqor, npos) match {
        case (Left(p), Left(np)) =>
            nq => np(p)
        case (_, Right(s)) =>
            nq => sir(s)
        case (Right(Left(q)), _) =>
            nq => nq(q)
        case (Right(Left(q)), Right(s)) =>
            nq => nq(q) // nq => sir(s)
        case (Right(Right(r)), _) =>
            nq => r
        case (_, Right(s)) =>
            nq => sir(s)
    }

$\{p \rightarrow q \vee \neg r, r \vee \neg q, \neg q \rightarrow r, r \rightarrow \neg q\} \vdash \neg p$

In [ ]:
def proof[P, Q, R](piqonr: P => Either[Q, Not[R]], 
                   ronq: Either[R, Not[Q]], 
                   nqir: Not[Q] => R, 
                   rinq: R => Not[Q]): Not[P] =
    p => ronq match {
        case Left(r) =>
            piqonr(p) match {
                case Left(q) =>
                    rinq(r)(q)
                case Right(nr) =>
                    nr(r)
            }
        case Right(nq) =>
            piqonr(p) match {
                case Left(q) =>
                    nq(q)
                case Right(nr) =>
                    nr(nqir(nq))
            }
    }

$\vdash (p \rightarrow q \wedge r) \wedge \neg q \rightarrow \neg p$

In [ ]:
def proof[P, Q, R]: ((P => (Q, R), Not[Q])) => Not[P] =
    {
        case (piqar, nq) =>
            p => nq(piqar(p)._1)
    }

$\{p \vee q \vee r, p \rightarrow s, \neg r \rightarrow \neg s\} \vdash \neg r \rightarrow q$

In [ ]:
def proof[P, Q, R, S](poqor: Either[P, Either[Q, R]], pis: P => S, nrims: Not[R] => Not[S]): Not[R] => Q =
    poqor match {
        case Left(p) =>
            nr => nrims(nr)(pis(p))
        case Right(Left(q)) =>
            nr => q
        case Right(Right(r)) =>
            nr => nr(r)
    }

## Ejercicios Tipos Algebraicos Datos

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $A$ y para todo tipo $B$: 

$(A + B)^2 \cong A^2 + 2 * A * B + B^2$ 

de tal forma que se verifique el siguiente test unitario para $A = B = Int$:

In [ ]:
class IsoTest(
    from: ((Either[Int, Int], Either[Int, Int])) => 
            Either[(Int, Int), Either[(Int, Int), Either[(Int, Int), (Int, Int)]]],
    to: Either[(Int, Int), Either[(Int, Int), Either[(Int, Int), (Int, Int)]]] => 
            (Either[Int, Int], Either[Int, Int])
) extends FlatSpec with Matchers { 
    "from-to" should "work" in {
        from(to(Left((1, 1)))) shouldBe Left((1, 1))
        from(to(Right(Left((1, 2))))) shouldBe Right(Left((1, 2)))
        from(to(Right(Right(Left((1, 2)))))) shouldBe Right(Right(Left((1, 2))))
        from(to(Right(Right(Right((2, 2)))))) shouldBe Right(Right(Right((2, 2))))
    }
    
    "to-from" should "work" in {
        to(from((Left(1), Left(1)))) shouldBe (Left(1), Left(1))
        to(from((Left(1), Right(2)))) shouldBe (Left(1), Right(2))
        to(from((Right(2), Left(1)))) shouldBe (Right(2), Left(1))
        to(from((Right(2), Right(2)))) shouldBe (Right(2), Right(2))
    }
}

In [ ]:
def from[A, B](l: (Either[A, B], Either[A, B])): Either[(A, A), Either[(A, B), Either[(A, B), (B, B)]]] = 
    l match {
        case (Left(a1), Left(a2)) =>
            Left((a1, a2))
        case (Left(a), Right(b)) =>
            Right(Left((a, b)))
        case (Right(b), Left(a)) =>
            Right(Right(Left(a, b)))
        case (Right(b1), Right(b2)) =>
            Right(Right(Right((b1, b2))))
    }

In [ ]:
def to[A, B](l: Either[(A, A), Either[(A, B), Either[(A, B), (B, B)]]]): (Either[A, B], Either[A, B]) = 
    l match {
        case Left((a1, a2)) =>
            (Left(a1), Left(a2))
        case Right(Left((a, b))) =>
            (Left(a), Right(b))
        case Right(Right(Left((a, b)))) =>
            (Right(b), Left(a))
        case Right(Right(Right((b1, b2)))) =>
            (Right(b1), Right(b2))
    }

In [ ]:
run(new IsoTest(from, to))

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$, para todo tipo $A$ y para todo tipo $B$: 

$(X + A) * (X + B) \cong X^2 + (A + B) * X + A * B$ 

de tal forma que se verifique el siguiente test unitario para $X = A = B = Boolean$:

In [ ]:
class IsoTest(
    from: ((Either[Boolean, Boolean], Either[Boolean, Boolean])) => 
            Either[(Boolean, Boolean), Either[(Boolean, Boolean), Either[(Boolean, Boolean), (Boolean, Boolean)]]],
    to: Either[(Boolean, Boolean), Either[(Boolean, Boolean), Either[(Boolean, Boolean), (Boolean, Boolean)]]] => 
            (Either[Boolean, Boolean], Either[Boolean, Boolean])
) extends FlatSpec with Matchers { 
    "from-to" should "work" in {
        from(to(Left(false, false))) shouldBe Left(false, false)
        from(to(Right(Right(Left((false, true)))))) shouldBe Right(Right(Left((false, true))))
        from(to(Right(Left((true, false))))) shouldBe Right(Left((true, false)))
        from(to(Right(Right(Right((true, true)))))) shouldBe Right(Right(Right((true, true))))
    }
    
    "to-from" should "work" in {
        to(from((Left(false), Left(false)))) shouldBe (Left(false), Left(false))
        to(from((Left(true), Right(true)))) shouldBe (Left(true), Right(true))
        to(from((Right(false), Left(true)))) shouldBe (Right(false), Left(true))
        to(from((Right(true), Right(false)))) shouldBe (Right(true), Right(false))
    }
}

In [ ]:
def from[X, A, B](l: (Either[X, A], Either[X, B])): Either[(X, X), Either[(A, X), Either[(B, X), (A, B)]]] =
    l match {
        case (Left(x1), Left(x2)) =>
            Left((x1, x2))
        case (Left(x), Right(b)) =>
            Right(Right(Left((b, x))))
        case (Right(a), Left(x)) =>
            Right(Left((a, x)))
        case (Right(a), Right(b)) =>
            Right(Right(Right((a, b))))
    }

In [ ]:
def to[X, A, B](l: Either[(X, X), Either[(A, X), Either[(B, X), (A, B)]]]): (Either[X, A], Either[X, B]) =
    l match {
        case Left((x1, x2)) =>
            (Left(x1), Left(x2))
        case Right(Right(Left((b, x)))) =>
            (Left(x), Right(b))
        case Right(Left((a, x))) =>
            (Right(a), Left(x))
        case Right(Right(Right((a, b)))) =>
            (Right(a), Right(b))
    }

In [ ]:
run(new IsoTest(from, to))